In [108]:
import numpy as np
import pandas as pd
from numpy import unique

In [109]:
def wrangle(filepath):
    df=pd.read_csv(filepath)
    drop_cols=[col for col in df.columns if "message_id" in col]
    drop_cols.append("customer_id")
    drop_cols.append("client_id")
    drop_cols.append("sent_at")
    drop_cols.append("created_at")
    drop_cols.append("message_number")
    df.drop(columns=drop_cols,inplace=True)
    return df

In [110]:
df = pd.read_csv("communications.csv")
print(df.shape)
print(df.head())

(145941, 15)
                         message_id                       customer_id  \
0  28d82c62db5989d9441351f58738c6a8  f13e966f9dc1e2aa7bffec6c9efa8e11   
1  0f98a1bbf7f2b66f420816a65c8d9b77  f13e966f9dc1e2aa7bffec6c9efa8e11   
2  97f9ed967d2334ce0b4ebfcfc9b5c25c  f13e966f9dc1e2aa7bffec6c9efa8e11   
3  3d0615ca493d69e16265ab688495c134  f13e966f9dc1e2aa7bffec6c9efa8e11   
4  3eb3a03a007bd1adf7482e6d95f7b942  f13e966f9dc1e2aa7bffec6c9efa8e11   

                            sent_at  clicked  converted  \
0  2021-01-04 10:03:16.376000+00:00        0          0   
1  2021-01-12 08:34:09.376000+00:00        0          0   
2  2021-01-19 08:58:11.376000+00:00        0          0   
3  2021-01-28 10:43:04.376000+00:00        0          0   
4  2021-02-04 11:28:56.376000+00:00        0          0   

                          client_id gender customer_country   age  \
0  cfcd208495d565ef66e7dff9f98764da      F              NaN  51.0   
1  cfcd208495d565ef66e7dff9f98764da      F             

In [111]:
counts=df.nunique()
print(counts)

message_id          145941
customer_id           9751
sent_at             145937
clicked                  2
converted                2
client_id                8
gender                   2
customer_country         3
age                     79
created_at            9751
client_name              8
product_type             3
client_country           3
channel                  2
message_number          39
dtype: int64


In [112]:
to_del=[i for i,v in enumerate(counts) if v<=1] #Delete Useless Column
print(df.columns[to_del])
df.drop(df.columns[to_del],axis=1,inplace=True)
counts=df.nunique()
print(counts)

Index([], dtype='object')
message_id          145941
customer_id           9751
sent_at             145937
clicked                  2
converted                2
client_id                8
gender                   2
customer_country         3
age                     79
created_at            9751
client_name              8
product_type             3
client_country           3
channel                  2
message_number          39
dtype: int64


In [113]:
dups=df.duplicated()
print(dups.any())
print(df[dups])


False
Empty DataFrame
Columns: [message_id, customer_id, sent_at, clicked, converted, client_id, gender, customer_country, age, created_at, client_name, product_type, client_country, channel, message_number]
Index: []


In [114]:
df.drop_duplicates(inplace=True)

In [115]:
#Count Missing Values
print(df.isnull().sum())
print(df.isnull().sum()* 100 / len(df))

message_id              0
customer_id             0
sent_at                 0
clicked                 0
converted               0
client_id               0
gender              31676
customer_country    30146
age                 22020
created_at              0
client_name             0
product_type            0
client_country          0
channel                 0
message_number          0
dtype: int64
message_id           0.000000
customer_id          0.000000
sent_at              0.000000
clicked              0.000000
converted            0.000000
client_id            0.000000
gender              21.704661
customer_country    20.656293
age                 15.088289
created_at           0.000000
client_name          0.000000
product_type         0.000000
client_country       0.000000
channel              0.000000
message_number       0.000000
dtype: float64


In [116]:
#Categorial Correlation using Chi-Square
subset = df[["gender","age","clicked","converted","product_type"]]

In [117]:
subset= wrangle("communications.csv")

In [118]:
subset.head()

,clicked,converted,gender,customer_country,age,client_name,product_type,client_country,channel
0,0,0,F,NaN,51.0,SwanPay,BNPL,UK,SMS
1,0,0,F,NaN,51.0,SwanPay,BNPL,UK,SMS
2,0,0,F,NaN,51.0,SwanPay,BNPL,UK,Email
3,0,0,F,NaN,51.0,SwanPay,BNPL,UK,Email
4,0,0,F,NaN,51.0,SwanPay,BNPL,UK,Email


In [119]:
def calculate_woe_iv(dataset, feature, target):
    lst = []
    for i in range(dataset[feature].nunique()):
        val = list(dataset[feature].unique())[i]
        
        lst.append({
            'Value': val,
            'All': dataset[dataset[feature] == val].count()[feature],
            'Good': dataset[(dataset[feature] == val) & (dataset[target] == 0)].count()[feature],
            'Bad': dataset[(dataset[feature] == val) & (dataset[target] == 1)].count()[feature]
        })
        
    dset = pd.DataFrame(lst)
    print(dset)
    dset['Distr_Good'] = dset['Good'] / dset['Good'].sum()
    dset['Distr_Bad'] = dset['Bad'] / dset['Bad'].sum()
    dset['WoE'] = np.log(dset['Distr_Good'] / dset['Distr_Bad'])
    dset = dset.replace({'WoE': {np.inf: 0, -np.inf: 0}})
    dset['IV'] = (dset['Distr_Good'] - dset['Distr_Bad']) * dset['WoE']
    iv = dset['IV'].sum()
    
    dset = dset.sort_values(by='WoE')
    
    return dset, iv

In [80]:
#Attribute Relevance Analysis
# IV Guideline
# <0.02 Useless
# 0.02-0.1 Weak
# 0.1-0.3 Medium
# 0.3-0.5 Strong
# >0.5 Too good to be true
data=subset
Target='converted'
for col in data.columns:
    if col == Target: continue
    else:
        print('WoE and IV for column: {}'.format(col))
        df, iv = calculate_woe_iv(data, col, Target)
        #print(df)
        print('IV score: {:.2f}'.format(iv))
        if iv<0.02:
            print("Result: Useless Indicator")
        elif iv>=0.02 and iv<0.1:
            print("Result: Weak Indicator")
        elif iv>=0.1 and iv<0.3:
            print("Result: Medium Indicator")
        elif iv>=0.3 and iv<0.5:
            print("Result: Strong Indicator")
        elif iv>=0.5:
            print("Result: Suspicious Indicator")
        print('\n')

WoE and IV for column: clicked
   Value     All    Good   Bad
0      0  136567  136567     0
1      1    9374    6232  3142
IV score: 3.00
Result: Suspicious Indicator


WoE and IV for column: gender
  Value    All   Good   Bad
0     F  56610  55207  1403
1     M  57655  56454  1201
IV score: 0.01
Result: Useless Indicator


WoE and IV for column: customer_country
  Value    All   Good  Bad
0   NaN      0      0    0
1    NZ  32738  32417  321
2    UK  49146  48156  990
IV score: 0.12
Result: Medium Indicator


WoE and IV for column: age
    Value    All   Good  Bad
0    51.0  11736  11543  193
1    28.0   3249   3135  114
2    24.0   4499   4324  175
3    26.0   3846   3707  139
4    22.0   4478   4306  172
..    ...    ...    ...  ...
74   77.0    472    469    3
75   90.0    223    223    0
76   94.0    100    100    0
77   80.0    328    328    0
78   93.0     85     84    1

[79 rows x 4 columns]
IV score: 0.36
Result: Strong Indicator


WoE and IV for column: client_name
        

In [120]:
# IV Guideline
# <0.02 Useless
# 0.02-0.1 Weak
# 0.1-0.3 Medium
# 0.3-0.5 Strong
# >0.5 Too good to be true
data=subset
Target='clicked'
for col in data.columns:
    if col == Target: continue
    else:
        print('WoE and IV for column: {}'.format(col))
        df, iv = calculate_woe_iv(data, col, Target)
        #print(df)
        print('IV score: {:.2f}'.format(iv))
        if iv<0.02:
            print("Result: Useless Indicator")
        elif iv>=0.02 and iv<0.1:
            print("Result: Weak Indicator")
        elif iv>=0.1 and iv<0.3:
            print("Result: Medium Indicator")
        elif iv>=0.3 and iv<0.5:
            print("Result: Strong Indicator")
        elif iv>=0.5:
            print("Result: Suspicious Indicator")
        print('\n')

WoE and IV for column: converted
   Value     All    Good   Bad
0      0  142799  136567  6232
1      1    3142       0  3142
IV score: 0.14
Result: Medium Indicator


WoE and IV for column: gender


C:\Users\USER\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


  Value    All   Good   Bad
0     F  56610  52663  3947
1     M  57655  53948  3707
IV score: 0.00
Result: Useless Indicator


WoE and IV for column: customer_country
  Value    All   Good   Bad
0   NaN      0      0     0
1    NZ  32738  30855  1883
2    UK  49146  46859  2287
IV score: 0.01
Result: Useless Indicator


WoE and IV for column: age
    Value    All   Good  Bad
0    51.0  11736  11107  629
1    28.0   3249   2973  276
2    24.0   4499   4071  428
3    26.0   3846   3501  345
4    22.0   4478   4050  428
..    ...    ...    ...  ...
74   77.0    472    452   20
75   90.0    223    218    5
76   94.0    100    100    0
77   80.0    328    318   10
78   93.0     85     83    2

[79 rows x 4 columns]
IV score: 0.14
Result: Medium Indicator


WoE and IV for column: client_name
          Value    All   Good   Bad
0       SwanPay  36110  33389  2721
1    Badgerbank  29007  28174   833
2  Eagle Energy   8103   7536   567
3   Moose Money  12396  10504  1892
4       KiwiPay  20876 

In [124]:
# IV Guideline
# <0.02 Useless
# 0.02-0.1 Weak
# 0.1-0.3 Medium
# 0.3-0.5 Strong
# >0.5 Too good to be true
data=subset
#data=data['channel'].replace(['SMS','Email'],[0,1],inplace=True)

Target='channel'
for col in data.columns:
    if col == Target: continue
    else:
        print('WoE and IV for column: {}'.format(col))
        df, iv = calculate_woe_iv(data, col, Target)
        #print(df)
        print('IV score: {:.2f}'.format(iv))
        if iv<0.02:
            print("Result: Useless Indicator")
        elif iv>=0.02 and iv<0.1:
            print("Result: Weak Indicator")
        elif iv>=0.1 and iv<0.3:
            print("Result: Medium Indicator")
        elif iv>=0.3 and iv<0.5:
            print("Result: Strong Indicator")
        elif iv>=0.5:
            print("Result: Suspicious Indicator")
        print('\n')

WoE and IV for column: clicked
   Value     All   Good    Bad
0      0  136567  50214  86353
1      1    9374   4878   4496
IV score: 0.02
Result: Weak Indicator


WoE and IV for column: converted
   Value     All   Good    Bad
0      0  142799  53866  88933
1      1    3142   1226   1916
IV score: 0.00
Result: Useless Indicator


WoE and IV for column: gender
  Value    All   Good    Bad
0     F  56610  21446  35164
1     M  57655  21749  35906
IV score: 0.00
Result: Useless Indicator


WoE and IV for column: customer_country
  Value    All   Good    Bad
0   NaN      0      0      0
1    NZ  32738  12441  20297
2    UK  49146  18384  30762
IV score: 0.00
Result: Useless Indicator


WoE and IV for column: age
    Value    All  Good   Bad
0    51.0  11736  4419  7317
1    28.0   3249  1265  1984
2    24.0   4499  1729  2770
3    26.0   3846  1458  2388
4    22.0   4478  1766  2712
..    ...    ...   ...   ...
74   77.0    472   182   290
75   90.0    223    67   156
76   94.0    100    